# Examining Racial Discrimination in the US Job Market

### Background
Racial discrimination continues to be pervasive in cultures throughout the world. Researchers examined the level of racial discrimination in the United States labor market by randomly assigning identical résumés to black-sounding or white-sounding names and observing the impact on requests for interviews from employers.

### Data
In the dataset provided, each row represents a resume. The 'race' column has two values, 'b' and 'w', indicating black-sounding and white-sounding. The column 'call' has two values, 1 and 0, indicating whether the resume received a call from employers or not.

Note that the 'b' and 'w' values in race are assigned randomly to the resumes when presented to the employer.

<div class="span5 alert alert-info">
### Exercises
You will perform a statistical analysis to establish whether race has a significant impact on the rate of callbacks for resumes.

Answer the following questions **in this notebook below and submit to your Github account**. 

   1. What test is appropriate for this problem? Does CLT apply?
   2. What are the null and alternate hypotheses?
   3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.
   4. Write a story describing the statistical significance in the context or the original problem.
   5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?

You can include written notes in notebook cells using Markdown: 
   - In the control panel at the top, choose Cell > Cell Type > Markdown
   - Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet


#### Resources
+ Experiment information and data source: http://www.povertyactionlab.org/evaluation/discrimination-job-market-united-states
+ Scipy statistical methods: http://docs.scipy.org/doc/scipy/reference/stats.html 
+ Markdown syntax: http://nestacms.com/docs/creating-content/markdown-cheat-sheet
+ Formulas for the Bernoulli distribution: https://en.wikipedia.org/wiki/Bernoulli_distribution
</div>
****

In [111]:
import pandas as pd
import numpy as np
from scipy import stats

In [112]:
data = pd.io.stata.read_stata('data/us_job_market_discrimination.dta')

In [113]:
# number of callbacks for black-sounding names
sum(data[data.race=='w'].call)

235.0

In [114]:
data.head()

,id,ad,education,ofjobs,yearsexp,honors,volunteer,military,empholes,occupspecific,...,compreq,orgreq,manuf,transcom,bankreal,trade,busservice,othservice,missind,ownership
0,b,1,4,2,6,0,0,0,1,17,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
1,b,1,3,3,6,0,1,1,0,316,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
2,b,1,4,1,6,0,0,0,0,19,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
3,b,1,3,4,6,0,1,0,1,313,...,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,
4,b,1,3,3,22,0,0,0,0,313,...,1.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,Nonprofit


<div class="span5 alert alert-success">
<p>Your answers to Q1 and Q2 here</p>
</div>

#### 1. What test is appropriate for this problem? Does CLT apply?

CLT applies here 

#### 2. What are the null and alternate hypotheses?
We are doing comparing two proportions where the proportion in question is call back rate. Our null hypothesis is that the black call back rate proportion, $p_b$, is the same as the white call back rate proportion, $p_w$. The alternative hypothesis is that the white call back rate proportion is greater than the black call back rate proportion.

H_0: $p_w$ = $p_b$

H_1: $p_w$ > $p_b$

$α$ = 0.05

In [115]:
# Race Specific data
w = data[data.race=='w']
b = data[data.race=='b']
combined = np.concatenate((w, b))

# Number of calls per race
w_calls = sum(data[data.race=='w'].call)
b_calls = sum(data[data.race=='b'].call)
    combined_calls = w_calls + b_calls

# Proportions
p_w = w_calls/len(w)
p_b = b_calls/len(b)
p_combined = combined_calls/len(combined)

print('white callback rate = %.3f   black callback rate = %.3f  combined callback rate = %.3f' % (p_w, p_b, p_combined))

white callback rate = 0.097   black callback rate = 0.064  combined callback rate = 0.080


#### 3. Compute margin of error, confidence interval, and p-value. Try using both the bootstrapping and the frequentist statistical approaches.

In [124]:
# Frequentist approach: Compute the p-value

# Standard Deviation
w_call_std = w.call.std()
b_call_std = b.call.std()

# Mean
w_call_mean = w.call.mean()
b_call_mean = b.call.mean()
call_mean = (w_call_mean + b_call_mean)/2

# length of array
white_n = len(w.call)
black_n = len(b.call)
n = white_n + black_n

# Empirical difference in the callback proportion of the two samples
empirical_diff_prop = p_w - p_b

# Calculate Z-statistic
Z = 1.96
std_error = np.sqrt((p_b*(1-p_b)/black_n)+(p_w*(1-p_w)/white_n))
z_statistic = (empirical_diff_prop)/std_error
p_val = stats.norm.sf(np.abs(z_statistic)) # one-tail p-value

print('z-statistic = %.3f   p-value = %.8f   standard error = %.4f' % (z_statistic, p_val, std_error))

z-statistic = 4.116   p-value = 0.00001931   standard error = 0.0078


In [125]:
# Frequentist approach with normal z-distribution: Margin of Error 95% confidence interval
Z = 1.96
error_margin = Z*std_error
freq_ztest_CI_95 = [round(empirical_diff_prop - error_margin, 3), round(empirical_diff_prop + error_margin,3)]

#print('95% Confidence Interval: ' + str(freq_ztest_CI_95))
print('95% Confidence Interval: = ({:2.3f}, {:2.3f})  Margin of error = {:.4f}'.format(freq_ztest_CI_95[0], freq_ztest_CI_95[1], error_margin))

95% Confidence Interval: = (0.017, 0.047)  Margin of error = 0.0153


In [185]:
# Perform a two-sample bootstrap hypothesis test

def get_prop_diff(data1, data2):
    p1 = np.sum(data1)/len(data1)
    p2 = np.sum(data2)/len(data2)
    
    return p1-p2

def get_permutation_replicate(data1, data2, func, size=1):
    """Generate a permutation sample from two data sets."""
    
    permutation_replicates = np.empty(size)
    data = np.concatenate((data1, data2))
    # Concatenate the data sets: data    
    for i in range(size):
        # Permute the concatenated array: permuted_data
        permutation_samples = np.random.permutation(data)

        # Split the permuted array into two: perm_sample_1, perm_sample_2
        perm_sample_1 = permutation_samples[:len(data1)]
        perm_sample_2 = permutation_samples[len(data1):]
        permutation_replicates[i] = func(perm_sample_1, perm_sample_2)
        
    return permutation_replicates


# Compute 10,000 bootstrap replicates from shifted arrays
bs_replicates = get_permutation_replicate(w.call, b.call, get_prop_diff, size=10000)

# Compute and print p-value: p
p_val = np.sum(bs_replicates >= empirical_diff_prop) / len(bs_replicates)
print('p-value = %.10f' % p_val)

p-value = 0.0001000000


In [186]:
# Bootstrap approach: 95% confidence interval
# Bootstrapping allows us to take percentiles of the bootstrap replicates to get the confidence interval
bootstrap_CI_95 = np.percentile(bs_replicates,[2.5,97.5])
bootstrap_CI_95 = [round(bootstrap_CI_95[0],3), round(bootstrap_CI_95[1],3)]
print('95% Confidence Interval: ' + str(bootstrap_CI_95))

95% Confidence Interval: [-0.015, 0.015]


<div class="span5 alert alert-success">
<p> Your answers to Q4 and Q5 here </p>
</div>

#### 4. Write a story describing the statistical significance in the context or the original problem.

#### 5. Does your analysis mean that race/name is the most important factor in callback success? Why or why not? If not, how would you amend your analysis?